# Septud recomendado



```
# importar librerias
%config IPCompleter.greedy=True
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter # para mostrar el porcentaje en el grafico
import geopandas as gpd
import os

import zipfile
import folium
from folium import plugins
from folium.plugins import Draw

from google.colab import files
from google.colab import drive

drive.mount('/content/drive')
```



# Funcion para Crear Marco con coordenadas UTM

In [ ]:
from pyproj import CRS, Transformer
import folium

def crear_mapa_con_marco_utm_dinamico(mapa_existente, lat_centro, lon_centro, zona_utm,
                                    ancho_marco=2000, intervalo=500,
                                    ancho_mapa=800, alto_mapa=600):
    """
    Versión que mantiene el diseño original del marco y añade dinamismo
    """
    # Configuración inicial de transformación
    crs_wgs84 = CRS("EPSG:4326")
    crs_utm = CRS(f"EPSG:326{zona_utm}")
    transformer = Transformer.from_crs(crs_wgs84, crs_utm, always_xy=True)
    centro_este, centro_norte = transformer.transform(lon_centro, lat_centro)

    # Generar HTML base manteniendo todos tus estilos
    marco_html = f"""
    <style>
        /* Mantener todos tus estilos CSS originales sin modificar */
        .map-container {{ position: relative; width: {ancho_mapa + 140}px; height: {alto_mapa + 140}px; padding: 20px; }}
        .utm-frame {{ position: relative; border: 2px solid #333; background: white; width: calc(100% - 80px); height: calc(100% - 80px); margin: 40px; }}
        .map-content {{ position: absolute; top: 50%; left: 50%; transform: translate(-50%, -50%); z-index: 1; width: {ancho_mapa}px; height: {alto_mapa}px; border: 1px solid #ddd; }}
        .coordenada {{ position: absolute; font-family: monospace; font-size: 14px; color: #333; background: rgba(255,255,255,0.9); padding: 4px; white-space: nowrap; z-index: 2; }}
        .linea-indicadora {{ position: absolute; background: #333; }}
        .coordenada-este.top {{ top: -34px; transform: translateX(-50%); }}
        .coordenada-este.top .linea-indicadora {{ width: 1px; height: 12px; top: 100%; left: 50%; }}
        .coordenada-este.bottom {{ bottom: -34px; transform: translateX(-50%); }}
        .coordenada-este.bottom .linea-indicadora {{ width: 1px; height: 12px; bottom: 100%; left: 50%; }}
        .coordenada-norte.left {{ left: -56px; transform: translateY(-50%); }}
        .coordenada-norte.left .texto-coordenada {{ display: inline-block; transform: rotate(-90deg) translateX(-8px); transform-origin: center; }}
        .coordenada-norte.left .linea-indicadora {{ width: 12px; height: 1px; left: 90%; top: 50%; }}
        .coordenada-norte.right {{ right: -52px; transform: translateY(-50%); }}
        .coordenada-norte.right .texto-coordenada {{ display: inline-block; transform: rotate(90deg); transform-origin: center; }}
        .coordenada-norte.right .linea-indicadora {{ width: 12px; height: 1px; right: calc(100% - 6px); top: 50%; }}
    </style>

    <div class="map-container">
        <div class="utm-frame" id="utm-frame">
            <!-- Las coordenadas se generarán dinámicamente aquí -->
        </div>

        <div class="map-content">
            {mapa_existente.get_root().render()}
        </div>
    </div>

    <!-- Incluir Proj4js para conversiones -->
    <script src="https://cdnjs.cloudflare.com/ajax/libs/proj4js/2.8.0/proj4.js"></script>

    <script>
        // Configurar proyección UTM
        proj4.defs("EPSG:326{zona_utm}",
            "+proj=utm +zone={zona_utm} +datum=WGS84 +units=m +no_defs");

        // Elementos clave
        const mapa = {mapa_existente.get_name()};
        const marco = document.getElementById('utm-frame');

        // Función de actualización mejorada
        const actualizarCoordenadas = () => {{
            // Obtener límites actuales del mapa
            const bounds = mapa.getBounds();
            const center = mapa.getCenter();

            // Convertir límites a UTM
            const ne_utm = proj4('EPSG:4326', 'EPSG:326{zona_utm}', [bounds.getEast(), bounds.getNorth()]);
            const sw_utm = proj4('EPSG:4326', 'EPSG:326{zona_utm}', [bounds.getWest(), bounds.getSouth()]);

            // Calcular dimensiones del marco
            const ancho_total = ne_utm[0] - sw_utm[0];
            const alto_total = ne_utm[1] - sw_utm[1];

            // Generar HTML para coordenadas
            let htmlCoords = '';

            // Coordenadas Este (X)
            for(let x = sw_utm[0] + {intervalo}; x < ne_utm[0]; x += {intervalo}) {{
                const percentX = ((x - sw_utm[0]) / ancho_total) * 100;
                htmlCoords += `
                    <div class="coordenada coordenada-este top" style="left:${{percentX}}%">
                        ${{Math.round(x)}}
                        <div class="linea-indicadora"></div>
                    </div>
                    <div class="coordenada coordenada-este bottom" style="left:${{percentX}}%">
                        ${{Math.round(x)}}
                        <div class="linea-indicadora"></div>
                    </div>
                `;
            }}

            // Coordenadas Norte (Y)
            for(let y = sw_utm[1] + {intervalo}; y < ne_utm[1]; y += {intervalo}) {{
                const percentY = ((y - sw_utm[1]) / alto_total) * 100;
                htmlCoords += `
                    <div class="coordenada coordenada-norte left" style="top:${{percentY}}%">
                        <span class="texto-coordenada">${{Math.round(y)}}</span>
                        <div class="linea-indicadora"></div>
                    </div>
                    <div class="coordenada coordenada-norte right" style="top:${{percentY}}%">
                        <span class="texto-coordenada">${{Math.round(y)}}</span>
                        <div class="linea-indicadora"></div>
                    </div>
                `;
            }}

            marco.innerHTML = htmlCoords;
        }};

        // Manejadores de eventos
        mapa.on('moveend', actualizarCoordenadas);
        mapa.on('zoomend', actualizarCoordenadas);

        // Actualización inicial
        actualizarCoordenadas();

        // Asegurar actualización después de la carga
        setTimeout(actualizarCoordenadas, 500);
    </script>
    """

    # Crear mapa final manteniendo dimensiones
    mapa_final = folium.Map(width=ancho_mapa+140, height=alto_mapa+140)
    mapa_final.get_root().html.add_child(folium.Element(marco_html))

    return mapa_final

# # Generar versión dinámica
# mapa = folium.Map(location=[6.25, -75.58], zoom_start=15)
# mapa_dinamico = crear_mapa_con_marco_utm_dinamico(
#     mapa_existente=mapa,
#     lat_centro=6.22204,
#     lon_centro=-75.61399,
#     zona_utm=18,
#     ancho_marco=1500,
#     intervalo=250
# )

# # Guardar y abrir
# mapa_dinamico

# Modificando la clase de Draw de folium y personalizando en codigo de javscript

In [ ]:
class CustomDraw(JSCSSMixin, MacroElement):
    _template = Template("""
        {% macro html(this, kwargs) %}
            <style>
                #export {
                    position: absolute;
                    top: 160px;
                    right: 20px;
                    z-index: 999;
                    background: white;
                    color: black;
                    padding: 6px;
                    border-radius: 4px;
                    font-family: 'Helvetica Neue';
                    cursor: pointer;
                    font-size: 12px;
                    text-decoration: none;
                }
            </style>
            <a href='#' id='export'>Export</a>
        {% endmacro %}

        {% macro script(this, kwargs) %}
            var options = {
                position: {{ this.position|tojson }},
                draw: {{ this.draw_options|tojson }},
                edit: {{ this.edit_options|tojson }},
            };

            var drawnItems_{{ this.get_name() }} = new L.featureGroup().addTo({{ this._parent.get_name() }});
            options.edit.featureGroup = drawnItems_{{ this.get_name() }};

            var {{ this.get_name() }} = new L.Control.Draw(options).addTo({{ this._parent.get_name() }});

            {{ this._parent.get_name() }}.on(L.Draw.Event.CREATED, function(e) {
                var layer = e.layer;
                var type = e.layerType;

                // Aplicar estilos personalizados
                if (type === 'polyline') {
                    layer.setStyle({
                        color: 'blue',
                        weight: 3,
                        opacity: 0.7
                    });
                    var distance = L.GeometryUtil.length(layer.getLatLngs());
                    layer.bindPopup('Distancia: ' + distance.toFixed(2) + ' metros');
                }

                if (type === 'marker') {
                    layer.setIcon(L.icon({
                        iconUrl: 'https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-2x-blue.png',
                        iconSize: [25, 41],
                        iconAnchor: [12, 41],
                        popupAnchor: [1, -34]
                    }));
                    var coords = layer.getLatLng();
                    layer.bindPopup('Coordenadas: ' + coords.lat.toFixed(4) + ', ' + coords.lng.toFixed(4));
                }

                if (type === 'rectangle') {
                    layer.setStyle({
                        color: 'green',
                        fillColor: 'yellow',
                        fillOpacity: 0.3,
                        weight: 2
                    });
                    var area = L.GeometryUtil.geodesicArea(layer.getLatLngs()[0]);
                    layer.bindPopup('Área: ' + (area/10000).toFixed(2) + ' hectáreas');
                }

                if (type === 'polygon') {
                    layer.setStyle({
                        color: 'red',
                        fillColor: 'yellow',
                        fillOpacity: 0.3,
                        weight: 2
                    });
                    var area = L.GeometryUtil.geodesicArea(layer.getLatLngs()[0]);
                    layer.bindPopup('Área: ' + (area/10000).toFixed(2) + ' hectáreas');
                }

                if (type === 'circle') {
                    layer.setStyle({
                        color: 'red',
                        fillColor: 'green',
                        fillOpacity: 0.3,
                        weight: 2
                    });
                    layer.bindPopup('Radio: ' + layer.getRadius().toFixed(2) + ' metros');
                }

                if (type === 'circlemarker') {
                    layer.setStyle({
                        color: 'blue',
                        fillColor: 'yellow',
                        fillOpacity: 0.6,
                        radius: 10
                    });
                    var coords = layer.getLatLng();
                    layer.bindPopup('Coordenadas: ' + coords.lat.toFixed(4) + ', ' + coords.lng.toFixed(4));
                }

                //Etiquetar el "shape" y guardar metadatos útiles
                layer.feature = layer.feature || {type: 'Feature', properties: {}};
                layer.feature.properties = layer.feature.properties || {};

                if (type === 'circle') {
                    layer.feature.properties.shape = 'circle';
                    layer.feature.properties.radius = layer.getRadius(); // metros
                } else if (type === 'circlemarker') {
                    layer.feature.properties.shape = 'circlemarker';
                    layer.feature.properties.radius_px = layer.getRadius(); // pixeles
                } else if (type === 'rectangle') {
                    layer.feature.properties.shape = 'rectangle';
                } else if (type === 'polygon') {
                    layer.feature.properties.shape = 'polygon';
                } else if (type === 'polyline') {
                    layer.feature.properties.shape = 'polyline';
                } else if (type === 'marker') {
                    layer.feature.properties.shape = 'marker';
                }

                drawnItems_{{ this.get_name() }}.addLayer(layer);
            });

            document.getElementById('export').onclick = function() {
                var fc = { type: 'FeatureCollection', features: [] };

                drawnItems_{{ this.get_name() }}.eachLayer(function(layer) {
                    var gj = layer.toGeoJSON();
                    gj.properties = gj.properties || {};

                    if (layer instanceof L.Circle) {
                        gj.properties.shape = 'circle';
                        gj.properties.radius = layer.getRadius(); // metros
                        // Garantiza que el geometry sea el centro (Point)
                        var c = layer.getLatLng();
                        gj.geometry = { type: 'Point', coordinates: [c.lng, c.lat] };
                    }
                    if (layer instanceof L.CircleMarker) {
                        gj.properties.shape = 'circlemarker';
                        gj.properties.radius_px = layer.getRadius(); // pixeles
                    }
                    if (layer instanceof L.Rectangle) {
                        gj.properties.shape = 'rectangle';
                    }
                    if (layer instanceof L.Polygon && !(layer instanceof L.Rectangle)) {
                        gj.properties.shape = 'polygon';
                    }
                    if (layer instanceof L.Polyline && !(layer instanceof L.Polygon)) {
                        gj.properties.shape = 'polyline';
                    }
                    if (layer instanceof L.Marker) {
                        gj.properties.shape = 'marker';
                    }

                    fc.features.push(gj);
                });

                var blob = new Blob([JSON.stringify(fc)], {type: 'application/json'});
                var url = window.URL.createObjectURL(blob);
                var a = document.createElement('a');
                a.href = url;
                a.download = {{ this.filename|tojson }};
                document.body.appendChild(a);
                a.click();
                document.body.removeChild(a);
            };

        {% endmacro %}
    """)

    default_js = [
        ('leaflet_draw_js', 'https://cdnjs.cloudflare.com/ajax/libs/leaflet.draw/1.0.4/leaflet.draw.js'),
        ('leaflet_geometry_util', 'https://cdn.jsdelivr.net/npm/leaflet-geometryutil@0.9.3/src/leaflet.geometryutil.min.js')
    ]

    default_css = [
        ('leaflet_draw_css', 'https://cdnjs.cloudflare.com/ajax/libs/leaflet.draw/1.0.4/leaflet.draw.css')
    ]

    def __init__(self, export=True, filename='datos.geojson', position='topleft',
                 draw_options=None, edit_options=None):
        super().__init__()
        self._name = 'CustomDraw'
        self.export = export
        self.filename = filename
        self.position = position
        self.draw_options = draw_options or {
            'circle': {'showRadius': True, 'metric': True},
            'circlemarker': {}
        }
        self.edit_options = edit_options or {'edit': True, 'remove': True}

In [ ]:
# ======================
# 1) Mapa con control de dibujo (CustomDraw)
# ======================
# Asumimos que YA definiste la clase CustomDraw tal como la compartiste.
# Si no, pega aquí tu clase CustomDraw.

from typing import Optional, Tuple
import folium
from IPython.display import display

def crear_mapa_con_dibujo(
    archivo_salida: str = 'mis_dibujos.geojson',
    centro: tuple = (19.4326, -99.1332),
    zoom: int = 12,
    m: Optional[folium.Map] = None,
    position: str = 'topleft',
    draw_options: dict = None,
    edit_options: dict = None,
    crear_feature_group: bool = False,
    nombre_feature_group: str = "Dibujos",
    mostrar: bool = True
) -> Tuple[folium.Map, object, Optional[folium.FeatureGroup], str]:
    """
    Crea (o reutiliza) un mapa Folium y agrega el control de dibujo `CustomDraw`.

    Parámetros
    ----------
    archivo_salida : str
        Nombre del archivo GeoJSON que descargará el botón 'Export'.
    centro : (lat, lon)
        Centro del mapa SI se crea uno nuevo. Se ignora si se pasa `m`.
    zoom : int
        Zoom inicial SI se crea mapa nuevo. Se ignora si se pasa `m`.
    m : folium.Map | None
        Mapa existente. Si es None, se crea uno nuevo.
    position : str
        Posición del control de dibujo ('topleft', 'topright', ...).
    draw_options : dict | None
        Opciones para Leaflet.draw (se usa un preset si es None).
    edit_options : dict | None
        Opciones de edición (por defecto {'edit': True, 'remove': True}).
    crear_feature_group : bool
        Si True, añade/reutiliza un FeatureGroup llamado `nombre_feature_group`.
        (Nota: el JS de CustomDraw usa su propio FeatureGroup interno para dibujar;
         este FG es solo para mantener compatibilidad con tu retorno/leyenda).
    nombre_feature_group : str
        Nombre del FeatureGroup opcional.
    mostrar : bool
        Si True, hace display del mapa y muestra el mensaje de exportación.

    Retorna
    -------
    (mapa, draw_control, feature_group, archivo_salida)
    """
    # 1) Reutilizar o crear el mapa
    if m is None:
        m = folium.Map(location=list(centro), zoom_start=zoom)
    elif not isinstance(m, folium.Map):
        raise TypeError("El parámetro 'm' debe ser un folium.Map o None.")

    # 2) (Opcional) crear/reutilizar un FeatureGroup "Dibujos"
    feature_group = None
    if crear_feature_group:
        # Buscar uno existente con el mismo nombre
        for child in m._children.values():
            if isinstance(child, folium.map.FeatureGroup) and getattr(child, 'layer_name', None) == nombre_feature_group:
                feature_group = child
                break
        if feature_group is None:
            feature_group = folium.FeatureGroup(name=nombre_feature_group)
            feature_group.add_to(m)

    # 3) Control de dibujo
    draw_options = draw_options or {
        'polyline':   {'shapeOptions': {'color': '#1f77b4'}},
        'polygon':    {'shapeOptions': {'color': '#d62728'}},
        'rectangle':  {'shapeOptions': {'color': '#2ca02c'}},
        'circle':     {'shapeOptions': {'color': '#ff7f0e'}},
        'circlemarker': {'shapeOptions': {'radius': 10}},
        'marker': {}
    }
    edit_options = edit_options or {'edit': True, 'remove': True}

    draw_control = CustomDraw(
        filename=archivo_salida,
        position=position,
        draw_options=draw_options,
        edit_options=edit_options
    )
    draw_control.add_to(m)

    # 4) Mostrar y mensaje
    if mostrar:
        display(m)
        print(
            f"📝 Dibuja en el mapa y haz clic en 'Export' (arriba a la derecha). "
            f"Se descargará '{archivo_salida}' a tu computador."
        )

    return m, draw_control, feature_group, archivo_salida

# ======================
# 2) Cargar el GeoJSON exportado al entorno Python
# ======================
def subir_geojson_colab():
    """
    (Solo Colab) Abre diálogo para subir el archivo GeoJSON descargado.
    Retorna el nombre del archivo subido.
    """
    if not EN_COLAB:
        raise RuntimeError("Esta función es solo para Google Colab.")
    print("📁 Sube aquí el GeoJSON exportado (p.ej., 'mis_dibujos.geojson'):")
    subidos = files.upload()
    nombre = next(iter(subidos.keys()))
    print(f"✅ Subido: {nombre}")
    return nombre

# ======================
# 3) Parseo de GeoJSON -> ee.Geometry.*
# ======================
import json
import ee

def geojson_a_geometrias_ee(geojson_path, default_circle_radius_m=None):
    with open(geojson_path, 'r') as f:
        data = json.load(f)

    features = data['features'] if data.get('type') == 'FeatureCollection' else data.get('features', [])
    geometries = []

    for feat in features:
        geom = feat.get('geometry', {})
        props = (feat.get('properties') or {})
        gtype = geom.get('type')
        coords = geom.get('coordinates')

        if gtype == 'Point':
            lng, lat = coords
            shape_tag = (props.get('shape') or '').lower()
            if shape_tag == 'circle' or ('radius' in props):
                radius = props.get('radius', default_circle_radius_m)
                if radius is None:
                    print("⚠️ Punto de círculo sin 'radius'. Dejo como Point.")
                    geometries.append(ee.Geometry.Point([lng, lat]))
                else:
                    geometries.append(ee.Geometry.Point([lng, lat]).buffer(float(radius)))
            else:
                geometries.append(ee.Geometry.Point([lng, lat]))

        elif gtype == 'LineString':
            geometries.append(ee.Geometry.LineString(coords))

        elif gtype == 'Polygon':
            geometries.append(ee.Geometry.Polygon(coords))

        elif gtype == 'MultiPolygon':
            geometries.append(ee.Geometry.MultiPolygon(coords))

        else:
            print(f"⚠️ Geometría no soportada: {gtype}")

    return geometries


# Utilidad interna para GeometryCollection (genera un FC temporal)
import os, tempfile
def _guardar_temp_featurecollection(features):
    tmp = tempfile.NamedTemporaryFile(delete=False, suffix='.geojson')
    fc = {"type": "FeatureCollection", "features": features}
    with open(tmp.name, 'w') as f:
        json.dump(fc, f)
    return tmp.name

# ======================
# 4) Visualización en geemap
# ======================
def visualizar_geometrias_en_geemap(geoms, center=(6.2518, -75.5636), zoom=8):
    """
    Añade una lista de ee.Geometry a un mapa geemap.
    """
    m = geemap.Map(center=[center[0], center[1]], zoom=zoom)
    for i, g in enumerate(geoms, start=1):
        m.addLayer(g, {'color': 'red'}, f'Geo {i}')
    display(m)
    return m

# ======================
# 5) Función “capa_con_dibujos” basada en GeoJSON
# ======================
def capa_con_dibujos(geojson_path):
    """
    Emula tu función: imprime info de las capas dibujadas,
    leyendo desde el GeoJSON exportado (no desde feature_group).
    """
    with open(geojson_path, 'r') as f:
        data = json.load(f)
    features = data['features'] if data.get('type') == 'FeatureCollection' else data.get('features', [])

    for i, feat in enumerate(features, start=1):
        gtype = feat['geometry']['type']
        props = feat.get('properties', {}) or {}
        if gtype == 'Point':
            coord = feat['geometry']['coordinates']
            r = props.get('radius', None)
            if r is not None:
                print(f"[{i}] Círculo (centro): {coord} - radio: {r} m")
            else:
                print(f"[{i}] Punto: {coord}")
        elif gtype == 'Polygon':
            print(f"[{i}] Polígono con {len(feat['geometry']['coordinates'][0])} vértices (anillo exterior).")
        elif gtype == 'LineString':
            print(f"[{i}] Línea con {len(feat['geometry']['coordinates'])} vértices.")
        elif gtype == 'MultiPolygon':
            print(f"[{i}] MultiPolígono con {len(feat['geometry']['coordinates'])} polígonos.")
        else:
            print(f"[{i}] Geometría: {gtype}")


# Funcion para subir una carpeta GIS a Folium

In [ ]:
def añadir_shape(mapa, zip_path="/content/drive/MyDrive/Sinu.zip", max_px=2048, opacidad_raster=0.70):
    """
    Carga y visualiza TODOS los .shp y .tif/.tiff contenidos en un archivo ZIP en un mapa Folium.

    - Extrae el ZIP a una carpeta temporal.
    - Añade cada SHP como GeoJson (CRS reproyectado a EPSG:4326 si es necesario).
    - Convierte cada GeoTIFF a PNG temporal (normalización + alfa) y lo dibuja con ImageOverlay.
    - Agrega control de capas.

    Parámetros
    ----------
    mapa : folium.Map
        Mapa objetivo.
    zip_path : str
        Ruta al archivo .zip con datos (por defecto: /content/drive/MyDrive/Sinu.zip).
    max_px : int
        Tamaño máximo del lado del raster para previsualización (reduce peso).
    opacidad_raster : float
        Opacidad del overlay de los rasters (0 a 1).
    """
    import os, zipfile, tempfile, shutil
    import numpy as np
    from PIL import Image
    import geopandas as gpd
    import folium
    import rasterio
    from rasterio.enums import Resampling
    from rasterio.warp import transform_bounds

    if not os.path.exists(zip_path):
        raise FileNotFoundError(f"No existe el ZIP: {zip_path}")

    # Carpeta temporal
    temp_dir = tempfile.mkdtemp(prefix="sinu_zip_")
    try:
        with zipfile.ZipFile(zip_path, 'r') as z:
            z.extractall(temp_dir)

        shp_paths, tif_paths = [], []
        for root, _, files in os.walk(temp_dir):
            for f in files:
                fl = f.lower()
                fp = os.path.join(root, f)
                if fl.endswith(".shp"):
                    shp_paths.append(fp)
                elif fl.endswith((".tif", ".tiff")):
                    tif_paths.append(fp)

        # --- 1) VECTORES: todos los .shp ---
        for shp in shp_paths:
            try:
                gdf = gpd.read_file(shp)  # GeoPandas soporta leer desde zip extraído y desde zip directamente
                if gdf.crs is not None and gdf.crs.to_string() != "EPSG:4326":
                    gdf = gdf.to_crs(4326)  # a lat/lon WGS84
                nombre = f"SHP: {os.path.splitext(os.path.basename(shp))[0]}"
                # Tooltip con primeras columnas no-geométricas
                campos = [c for c in gdf.columns if c != "geometry"][:5] or None
                folium.GeoJson(
                    gdf,
                    name=nombre,
                    tooltip=folium.GeoJsonTooltip(fields=campos) if campos else None,
                ).add_to(mapa)
            except Exception as e:
                print(f"[SHP] No se pudo cargar {shp}: {e}")

        # --- 2) RASTERS: todos los .tif/.tiff ---
        for tif in tif_paths:
            try:
                with rasterio.open(tif) as src:
                    # Bounds -> WGS84 (lat/lon) para ImageOverlay de Folium/Leaflet
                    b = transform_bounds(src.crs, "EPSG:4326", *src.bounds, densify_pts=21)

                    # Leemos banda 1 (o componemos RGB si tienes multibanda; aquí normalizamos 1 banda a gris)
                    # Reescalado a máx max_px en el lado mayor
                    scale = min(1.0, float(max_px) / max(src.width, src.height))
                    out_h, out_w = int(src.height * scale), int(src.width * scale)
                    band1 = src.read(
                        1,
                        out_shape=(out_h, out_w) if scale < 1.0 else None,
                        resampling=Resampling.bilinear if scale < 1.0 else Resampling.nearest
                    ).astype("float32")

                    # Enmascarar nodata
                    nodata = src.nodata
                    mask = np.zeros_like(band1, dtype=bool)
                    if nodata is not None:
                        mask |= (band1 == nodata)

                    # Normalización robusta (2–98 percentil)
                    vals = band1[~mask]
                    if vals.size == 0:
                        print(f"[TIFF] {tif} sin datos útiles; se omite.")
                        continue
                    vmin, vmax = np.percentile(vals, [2, 98]).astype("float32")
                    if vmax <= vmin:
                        vmax = vmin + 1.0
                    scaled = np.clip((band1 - vmin) / (vmax - vmin), 0, 1)
                    gray = (scaled * 255).astype("uint8")

                    # Alfa: transparente donde nodata
                    alpha = (~mask).astype("uint8") * int(opacidad_raster * 255)
                    rgba = np.dstack([gray, gray, gray, alpha])

                    # PNG temporal
                    png_path = os.path.join(temp_dir, os.path.basename(tif) + ".png")
                    Image.fromarray(rgba, mode="RGBA").save(png_path)

                    # Agregar overlay
                    nombre = f"TIFF: {os.path.basename(tif)}"
                    folium.raster_layers.ImageOverlay(
                        name=nombre,
                        image=png_path,
                        bounds=[[b[1], b[0]], [b[3], b[2]]],  # [[south, west], [north, east]]
                        opacity=1.0,
                        interactive=True,
                        cross_origin=False
                    ).add_to(mapa)
            except Exception as e:
                print(f"[TIFF] No se pudo cargar {tif}: {e}")

        # Control de capas
        folium.LayerControl(collapsed=False).add_to(mapa)
        print("¡Capas del ZIP añadidas correctamente!")
    finally:
        # Limpieza: si prefieres conservar los PNG, comenta esta línea
        shutil.rmtree(temp_dir, ignore_errors=True)